In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('train_data.csv')
df2 = pd.read_csv('test_data.csv')

print(df.shape)
print(df2.shape)

(42000, 785)
(28000, 784)


In [3]:
x = df.drop('label',axis=1)
y = df.label

print(x.shape)
print(y.shape)

(42000, 784)
(42000,)


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x = scaler.fit_transform(x)
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
x = x.reshape((42000,28,28,1))
x.shape

(42000, 28, 28, 1)

In [6]:
x1 = np.array(df2)
x1 = scaler.fit_transform(x1)
x1 = x1.reshape((28000,28,28,1))
x1.shape

(28000, 28, 28, 1)

In [7]:
y = np.array(y)

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
y= y.reshape((-1,1))
y = enc.fit_transform(y)

y.shape

C:\Users\PRE-SETUP\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(42000, 10)

In [8]:
from sklearn.model_selection import train_test_split as tts

x_train,x_test,y_train,y_test = tts(x,y,test_size = 0.2, random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(33600, 28, 28, 1)
(33600, 10)
(8400, 28, 28, 1)
(8400, 10)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [10]:
model = keras.Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [11]:
from tensorflow import keras

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-5,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=25,
        verbose=1)
]

In [ ]:
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=["accuracy"])

history = model.fit(x,y,batch_size=64,epochs=20,validation_data=(x_test,y_test))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.figure()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

Epoch 1/20
657/657 [==============================] - 1243s 2s/step - loss: 0.3207 - accuracy: 0.8980 - val_loss: 0.0595 - val_accuracy: 0.9840
Epoch 2/20
657/657 [==============================] - 1117s 2s/step - loss: 0.0952 - accuracy: 0.9735 - val_loss: 0.0318 - val_accuracy: 0.9905
Epoch 3/20
657/657 [==============================] - 742s 1s/step - loss: 0.0751 - accuracy: 0.9790 - val_loss: 0.0271 - val_accuracy: 0.9911
Epoch 4/20
657/657 [==============================] - 709s 1s/step - loss: 0.0597 - accuracy: 0.9835 - val_loss: 0.0203 - val_accuracy: 0.9939
Epoch 5/20
657/657 [==============================] - 378s 575ms/step - loss: 0.0509 - accuracy: 0.9856 - val_loss: 0.0165 - val_accuracy: 0.9948
Epoch 6/20
657/657 [==============================] - 374s 570ms/step - loss: 0.0485 - accuracy: 0.9865 - val_loss: 0.0166 - val_accuracy: 0.9945
Epoch 7/20
657/657 [==============================] - 376s 572ms/step - loss: 0.0417 - accuracy: 0.9885 - val_loss: 0.0147 - val_accur

In [19]:
predictions=model.predict(x1)
pre=predictions.argmax(axis=-1)

In [20]:
submission = pd.Series(pre,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),submission],axis = 1)
submission.to_csv("final_submission_v1.csv",index=False)
submission.head()

ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      0
4        5      3

In [21]:
prediction=model.predict(x)
pred=prediction.argmax(axis=-1)